# Insect Classification Pipeline with Bplusplus

This notebook demonstrates a complete pipeline for detecting and classifying insects in videos using the Bplusplus library.

## What you'll accomplish:
1. **Collect** insect images from GBIF database
2. **Prepare** data by detecting and cropping insects from raw images  
3. **Train** a classification model for family, genus, and species identification
4. **Test** the trained model's performance
5. **Run inference** on videos to detect, classify, and track insects over time

## How it works:
- Uses a **two-stage approach**: pre-trained detection + custom classification
- Detection stage: Locates insects in images/video frames
- Classification stage: Identifies insects at three taxonomic levels (family → genus → species)
- Tracking stage: Follows insects over time and aggregates predictions per individual

By the end, you'll have a system that can process videos and output the most likely classification for each tracked insect.



## Setup: Create Virtual Environment (Recommended)

Create an isolated environment to avoid package conflicts:

```bash
python3 -m venv bplusplus_env
source bplusplus_env/bin/activate
```

## Setup: Install Required Packages

In [ ]:
! pip install bplusplus

## Import required packages

In [1]:
import bplusplus
from typing import Any
from pathlib import Path
import requests
from tqdm import tqdm

## Set directories

In [2]:
MAIN_DIR = Path("./")

GBIF_DATA_DIR = MAIN_DIR / "GBIF_data"
PREPARED_DATA_DIR = MAIN_DIR / "prepared_data"
TRAINED_MODEL_DIR = MAIN_DIR / "trained_model"

## Step 1: Collect Insect Images from GBIF

We download images from the GBIF (Global Biodiversity Information Facility) database for our target species.

**Important notes:**
- Download more images than needed - many will be filtered out during preparation
- Internet connection may be unstable - monitor progress and resume if needed
- Check the `GBIF_DATA_DIR` folder to track downloaded files

In [3]:
names = [
        "Coccinella septempunctata", "Apis mellifera", "Bombus lapidarius"
    ]

search: dict[str, Any] = {
    "scientificName": names
}

In [5]:
bplusplus.collect(
    group_by_key=bplusplus.Group.scientificName,
    search_parameters=search, 
    images_per_group=100,
    output_directory=GBIF_DATA_DIR,
    num_threads=3
)


Thread 0 starting collection for 1 species.
Creating folders for images...
Thread 1 starting collection for 1 species.
Creating folders for images...
Beginning to collect images from GBIF...
Thread 2 starting collection for 1 species.
Creating folders for images...
Beginning to collect images from GBIF...
Beginning to collect images from GBIF...


Finished collecting images.
Thread 2 finished collection.


KeyboardInterrupt: 







Traceback (most recent call last):
  File "/mnt/f/datasets/practice/env/lib/python3.10/site-packages/tqdm/std.py", line 1191, in __iter__
      File "/mnt/f/datasets/practice/env/lib/python3.10/site-packages/tqdm/std.py", line 1242, in update
      File "/mnt/f/datasets/practice/env/lib/python3.10/site-packages/tqdm/std.py", line 1347, in refresh
    self.display()
  File "/mnt/f/datasets/practice/env/lib/python3.10/site-packages/tqdm/std.py", line 1494, in display
    self.moveto(pos)
  File "/mnt/f/datasets/practice/env/lib/python3.10/site-packages/tqdm/std.py", line 1443, in moveto
    self.fp.write('\n' * n + _term_move_up() * -n)
  File "/mnt/f/datasets/practice/env/lib/python3.10/site-packages/tqdm/utils.py", line 196, in inner
    return func(*args, **kwargs)
  File "/mnt/f/datasets/practice/env/lib/python3.10/site-packages/ipykernel/iostream.py", line 694, in write
    self._schedule_flush()
  File "/mnt/f/datasets/practice/env/lib/python3.10/site-packages/ipykernel/iostr

## Step 2: Prepare Data for Training

This step uses a pre-trained vision model to:
1. **Detect** insects in the raw GBIF images
2. **Crop** each detected insect to focus on the subject
3. **Resize** images to a consistent size for training

**What to expect:**
- Many images will be rejected (low success rate is normal)
- Only clear, well-detected insects proceed to training
- This filtering ensures high-quality training data

In [6]:
bplusplus.prepare(
    input_directory=GBIF_DATA_DIR,
    output_directory=PREPARED_DATA_DIR,
    img_size=60
)

STARTING BPLUSPLUS DATASET PREPARATION
Input directory: GBIF_data
Output directory: prepared_data
Target image size: 60px (smallest dimension)

Using temporary directory: /tmp/tmp554hyz6q

STEP 1: Setting up directories and copying images...
--------------------------------------------------
  Created temporary images directory: /tmp/tmp554hyz6q/images
  Scanning input directory for class folders...
  Found 3 class folders
  Copying 71 images from class 'Apis mellifera'...
    ✓ 71 images copied for class 'Apis mellifera'
  Copying 100 images from class 'Bombus lapidarius'...
    ✓ 100 images copied for class 'Bombus lapidarius'
  Copying 75 images from class 'Coccinella septempunctata'...
    ✓ 75 images copied for class 'Coccinella septempunctata'
  Total images in temporary directory: 246
✓ Step 1 completed: 246 images copied from 3 classes

STEP 2: Cleaning images and setting up YOLO model...
--------------------------------------------------
  Checking for corrupted images...
  ✓ 

KeyboardInterrupt: 

## Step 3: Train the Classification Model

Train a ResNet-based neural network that classifies insects at three taxonomic levels:
- **Family** (e.g., Coccinellidae for ladybugs)
- **Genus** (e.g., Coccinella)  
- **Species** (e.g., Coccinella septempunctata)

The model learns hierarchical relationships between these classification levels. 

In [8]:
bplusplus.train(
    batch_size=4,
    epochs=9,
    patience=3,
    img_size=60,
    data_dir=PREPARED_DATA_DIR,
    output_dir=TRAINED_MODEL_DIR,
    species_list=names
)


INFO:Hyperparameters - Batch size: 4, Epochs: 9, Patience: 3, Image size: 60, Data directory: prepared_data, Output directory: trained_model
INFO:Using 3 species in the specified order
INFO:Building taxonomy from GBIF for 3 species



Taxonomy Results:
--------------------------------------------------------------------------------
Species                        Family               Genus                Status
--------------------------------------------------------------------------------
Coccinella septempunctata      Coccinellidae        Coccinella           OK


INFO:Taxonomy built: 2 families, 3 genera, 3 species
INFO:Dataset loaded with 91 valid images
INFO:Dataset loaded with 11 valid images
INFO:Initializing model...


Apis mellifera                 Apidae               Apis                 OK
Bombus lapidarius              Apidae               Bombus               OK
--------------------------------------------------------------------------------

Family indices:
  0: Apidae
  1: Coccinellidae

Genus indices:
  0: Coccinella
  1: Apis
  2: Bombus

Species indices:
  0: Coccinella septempunctata
  1: Apis mellifera
  2: Bombus lapidarius


INFO:Model structure initialized with 26659400 parameters
INFO:Setting up loss function and optimizer...
INFO:Testing model with a dummy input...
INFO:Forward pass test successful, output shapes: [torch.Size([1, 2]), torch.Size([1, 3]), torch.Size([1, 3])]
INFO:Starting training process...
INFO:Starting training
Epoch 1/9 [Valid]: 100%|██████████| 3/3 [00:00<00:00, 19.05it/s, loss=2.2823]
INFO:Saved best model at epoch 1 with validation loss: 2.1313


...

Epoch 8/9 [Valid]: 100%|██████████| 3/3 [00:00<00:00, 18.36it/s, loss=0.1451]
INFO:No improvement for 1 epochs. Best val loss: 0.5280



Epoch 8/9
Train Loss: 0.6471 (CE: 1.2382, Dep: 0.0560)
Valid Loss: 0.5814
Level 1 - Train Acc: 0.8791, Valid Acc: 1.0000, Unsure: 0.0000
Level 2 - Train Acc: 0.8242, Valid Acc: 0.7273, Unsure: 0.0000
Level 3 - Train Acc: 0.8242, Valid Acc: 0.7273, Unsure: 0.0909
------------------------------------------------------------


Epoch 9/9 [Valid]: 100%|██████████| 3/3 [00:00<00:00, 17.97it/s, loss=0.1072]
INFO:No improvement for 2 epochs. Best val loss: 0.5280
INFO:Training completed successfully
INFO:Model saved successfully with taxonomy information!



Epoch 9/9
Train Loss: 0.7650 (CE: 1.4179, Dep: 0.1121)
Valid Loss: 0.6231
Level 1 - Train Acc: 0.9121, Valid Acc: 1.0000, Unsure: 0.0000
Level 2 - Train Acc: 0.7802, Valid Acc: 0.8182, Unsure: 0.0000
Level 3 - Train Acc: 0.7912, Valid Acc: 0.7273, Unsure: 0.1818
------------------------------------------------------------
Model saved successfully with taxonomy information!


(HierarchicalInsectClassifier(
   (backbone): ResNet(
     (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
     (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (relu): ReLU(inplace=True)
     (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
     (layer1): Sequential(
       (0): Bottleneck(
         (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
         (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
         (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
         (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
         (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
         (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
         (relu): ReLU(inplace=True)
       

## Step 4: Download Pre-trained Detection Weights

For the two-stage pipeline, we need pre-trained weights for insect detection.

**Option 1:** Manual download from:
https://github.com/Tvenver/Bplusplus/releases/download/v1.2.3/v11small-generic.pt

**Option 2:** Automatic download using the code below:


In [9]:
def __download_file_from_github_release(url, dest_path):

    """
    Downloads a file from a given GitHub release URL and saves it to the specified destination path,
    with a progress bar displayed in the terminal.

    Args:
        url (str): The URL of the file to download.
        dest_path (Path): The destination path where the file will be saved.

    Raises:
        Exception: If the file download fails.
    """

    response = requests.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    block_size = 1024  # 1 Kibibyte
    progress_bar = tqdm(total=total_size, unit='iB', unit_scale=True)

    if response.status_code == 200:
        with open(dest_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=block_size):
                progress_bar.update(len(chunk))
                f.write(chunk)
        progress_bar.close()
    else:
        progress_bar.close()
        raise Exception(f"Failed to download file from {url}")

In [4]:
YOLO_WEIGHTS = TRAINED_MODEL_DIR / "v11small-generic.pt"

In [11]:
github_release_url = 'https://github.com/Tvenver/Bplusplus/releases/download/v1.2.3/v11small-generic.pt'

if not YOLO_WEIGHTS.exists():
    __download_file_from_github_release(github_release_url, YOLO_WEIGHTS)

100%|██████████| 38.1M/38.1M [00:02<00:00, 16.3MiB/s]


## Step 5: Test the Trained Model

Evaluate your trained model on a test dataset to measure accuracy.

**Test data requirements:**
- Directory structure: `test_data/images/` and `test_data/labels/`
- Images: Standard image formats (jpg, png, etc.)
- Labels: YOLO format text files with bounding boxes
- Label format: `<class> <x_center> <y_center> <width> <height>`
- Class names must match your training species order

**Note:** This tests detection + classification accuracy, but does not include tracking.


In [5]:
TEST_DATA_DIR = MAIN_DIR / "test_data" #if you want to test the model on a different dataset (two stage)
RESNET_MULTITASK_WEIGHTS = TRAINED_MODEL_DIR / "best_multitask.pt"

In [13]:
bplusplus.test(
    species_list=names,
    test_set=TEST_DATA_DIR,
    yolo_weights=YOLO_WEIGHTS,
    hierarchical_weights=RESNET_MULTITASK_WEIGHTS,
    output_dir=TRAINED_MODEL_DIR
)


INFO:Found 1 CUDA device(s)
INFO:GPU 0: NVIDIA GeForce RTX 3070 Ti Laptop GPU with 8.23 GB memory
INFO:CUDA initialization successful
INFO:Using device: cuda:0
INFO:Results will be saved to: trained_model
INFO:Loading model from trained_model/best_multitask.pt
INFO:Model loaded to CPU successfully
INFO:Building taxonomy from GBIF for 3 species


Using device: cuda:0
Using taxonomy from saved model
Saved model was trained on: Coccinella septempunctata, Apis mellifera, Bombus lapidarius

Taxonomy Results:
--------------------------------------------------------------------------------
Species                        Family               Genus                Status
--------------------------------------------------------------------------------
Coccinella septempunctata      Coccinellidae        Coccinella           OK
Apis mellifera                 Apidae               Apis                 OK


INFO:Taxonomy built: 2 families, 3 genera, 3 species


Bombus lapidarius              Apidae               Bombus               OK
--------------------------------------------------------------------------------

Family indices:
  0: Apidae
  1: Coccinellidae

Genus indices:
  0: Coccinella
  1: Apis
  2: Bombus

Species indices:
  0: Coccinella septempunctata
  1: Apis mellifera
  2: Bombus lapidarius
Using model with class counts: [2, 3, 3]


/mnt/f/datasets/practice/env/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/mnt/f/datasets/practice/env/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Model weights loaded successfully
Model successfully transferred to cuda:0
Model successfully loaded
Using species: Coccinella septempunctata, Apis mellifera, Bombus lapidarius


Processing Images: 100%|██████████| 2854/2854 [01:06<00:00, 42.60image/s]


Results saved to trained_model/results_hierarchical_20250730_145227.csv

Total time: 67.00 seconds
Average time per frame: 0.0235 seconds

Using the following index mappings for evaluation:

Family indices:
  0: Apidae
  1: Coccinellidae

Genus indices:
  0: Coccinella
  1: Apis
  2: Bombus

Species indices:
  0: Coccinella septempunctata
  1: Apis mellifera
  2: Bombus lapidarius
Prediction counts: {'true_species_boxes': 3257, 'true_genus_boxes': 2212, 'true_family_boxes': 2212, 'predicted_species': 3396, 'predicted_genus': 3396, 'predicted_family': 3396}

=== Species-level Metrics ===
+------------------------------+-------------+----------+------------+-----------+
| Label                        |   Precision |   Recall |   F1 Score |   Support |
+==============================+=============+==========+============+===========+
| Apis mellifera               |        0    |     0    |       0    |      1663 |
+------------------------------+-------------+----------+------------+----

## Step 6: Run Video Inference

Process a video to detect, classify, and track insects over time.

**How it works:**
1. **Detection:** Finds insects in each video frame
2. **Classification:** Identifies each detected insect 
3. **Tracking:** Follows individual insects across frames
4. **Aggregation:** Combines predictions for each tracked insect

**Key parameters:**
- `fps`: Video processing frame rate (None = use original)
- `tracker_max_frames`: How long to remember lost insects
  - Example: 60 frames at 15 fps = 4 seconds before forgetting an insect

In [6]:
VIDEO_INPUT_PATH = MAIN_DIR / 'videos' / "test_video.mp4"
VIDEO_OUTPUT_PATH = MAIN_DIR / 'videos' / "test_video_output.mp4"

In [9]:
bplusplus.inference(
    species_list=names,
    yolo_model_path=YOLO_WEIGHTS,
    hierarchical_model_path=RESNET_MULTITASK_WEIGHTS,
    confidence_threshold=0.35,
    video_path=VIDEO_INPUT_PATH,
    output_path=VIDEO_OUTPUT_PATH,
    tracker_max_frames=200,
    fps=None
    )

INFO:Building taxonomy from GBIF for 3 species


Processing single video: videos/test_video.mp4
Initializing models...
Using device: cuda

Species                        Family               Genus                Status
--------------------------------------------------------------------------------


INFO:Taxonomy built: 2 families, 3 genera, 3 species


Coccinella septempunctata      Coccinellidae        Coccinella           OK
Apis mellifera                 Apidae               Apis                 OK
Bombus lapidarius              Apidae               Bombus               OK
--------------------------------------------------------------------------------

Family indices:
  0: Apidae
  1: Coccinellidae

Genus indices:
  0: Coccinella
  1: Apis
  2: Bombus

Species indices:
  0: Coccinella septempunctata
  1: Apis mellifera
  2: Bombus lapidarius
Loading YOLO model from trained_model/v11small-generic.pt
Loading hierarchical model from trained_model/best_multitask.pt
Model architecture: [2, 3, 3] classes per level
Models loaded successfully!

Processing: videos/test_video.mp4
Processing video: videos/test_video.mp4
Properties: 1793 frames, 30.00 FPS, 59.77s duration
Output video: videos/test_video_output.mp4 at 30.00 FPS
Processed: 25/1793 frames (1.4%)
Processed   1.47s | Track 95426431 | Coccinella septempunctata (46.7%)
Processed   

{'video_file': 'test_video.mp4',
 'output_path': PosixPath('videos/test_video_output.mp4'),
 'success': True,
 'detections': 115,
 'tracks': 2,
 'error': None}